Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [2]:
NAME = "Camille Hascoët"

---

In [3]:
import IPython
assert IPython.version_info[0] >= 3, "Your version of Jupyter is too old, please update it."

import numpy as np
from numpy.random import default_rng
from typing import List, Set, Dict, Tuple
from numpy.testing import assert_approx_equal, assert_allclose
rng = default_rng(2023) # common random number generator

from sklearn.model_selection import train_test_split
from sklearn.datasets import make_blobs, make_classification, make_moons


# Assignment: $k$-Fold Cross-Validation

In this assignment, you will implement a function that compares two learning algorithms using $k$-fold cross-validation and apply it to compare some simple learning algorithms.

**Tasks:**
1. Implement class `Perceptron` for the perceptron learning algorithm. **(2 points)**
2. Implement function `cross_val` for evaluating the difference of errors between two given learning algorithms using $k$-fold cross-validation. **(4 points)**
3. Implement function `conf_interval` for computing confidence interval for an error difference when using $k$-fold cross-validation. **(1 point)**
4. By applying the functions implemented in the above tasks, compare several learning algorithms. **(3 points)**

This notebook was generated using `nbgrader`. It has a special format. Several cells contain tests. Some of the tests are hidden. Please, **do not break the format**:
1. do not delete cells where you should insert your answers (code or text), and
2. do not copy complete cells (you can freely copy the contents of cells).
Otherwise, you can add and delete cells. 

A learning algorithm tries to learn a target function $f: \mathrm{R}^n \to \{0,1\}$, where $\mathrm{R}$ is the set of real numbers. The goal of a learning algorithm is to identify a function $h: \mathrm{R}^n \to \{0,1\}$, called *hypothesis*, from some class of functions $\mathcal{H}$ such that the function $h$ is a good approximation of the target function $f$. The only information the algorithm can use is a sample $S\subset X$ called a *training set*, and the correct value $f(x)$, called *label*, for all $x \in S$. The sample $S$ is a set of $n$ elements from $X$ randomly selected according to some probabilistic distribution $D$.

We suppose that each learning algorithm is implemented as a subclass of the following 
class `LearningAlgorithm`:

In [4]:
class LearningAlgorithm:
    """ Base class for learning algorithms. """

    def __init__(self, **learning_par):
        self.par = learning_par

    def fit(self, X: np.ndarray, y: np.ndarray) -> None:
        """ Learn the parameters of the algorithm. 
        The learned parameters are stored into member variables.
        """
        raise NotImplementedError
        
    def predict(self, X: np.ndarray) -> np.array:
        """ Apply the learned function to the input vectors `X`. 
        The returned value is a vector of the results - zeros and ones.
        """
        raise NotImplementedError
        
    def score(self, X: np.ndarray, y: np.ndarray) -> float:
        """ Compute the mean accuracy on the test vectors `X`, where `y` contains the 
        correct labels for the vectors (the rows) in `X`. 
        """
        output = self.predict(X)
        return np.mean(output == y)

where

Param      |Meaning
--------------|---------------------------------------------------------------
`learning_par`|is a dictionary of parameters of the learning algorithm,
`fit`         |is the learning function; it computes learned parameters and stores them into member variables; it can use the parameters from the member variable `par`,
`X`           |is a two-dimensional numpy array (training and test vectors are the rows of `X`),
`y`           |is a vector of desired labels (zeros and ones), and
`predict`     |computes the learned function for the input vectors from a two-dimensional numpy array `X` (each row of the array is an input vector); it uses the learned parameters and/or the parameters from the member variable `par`; the returned value is a vector of the results - zeros and ones,
`score`       |computes the mean accuracy of the trained algorithm on the test vectors X, where y contains the correct labels for the vectors (the rows) in X.

## A simple learning algorithm `Memorizer`

Below, we implement a simple learning algorithm called `Memorizer` that memorizes all training samples and their true labels. Trained `Memorizer` answers 
* correctly on the inputs from the training set, and 
* randomly otherwise - it outputs zeroes and ones with the same ratio as in the training set.

In [5]:
class Memorizer(LearningAlgorithm):
    
    def __init__(self, rng_seed=42):
        super().__init__(_seed=rng_seed, _rng=default_rng(rng_seed))
        
    def fit(self, X: np.ndarray, y: np.ndarray) -> None:
        self._learned_par = [np.atleast_2d(X), y]
    
    def predict(self, X: np.ndarray) -> np.array:
        # generate random outputs
        # the following line of code ensures that each call of predict on the same `X` 
        # will return the same output; for a "serious" application, this line should be omitted
        self.par['_rng'] = default_rng(self.par['_seed'])
        
        X = np.atleast_2d(X)
        out = self.par['_rng'].binomial(1, self._learned_par[1].mean(), X.shape[0])
        for i in range(X.shape[0]):
            res = (self._learned_par[0] == X[i]).all(axis=1).nonzero()[0]
            if res.size > 0:
                out[i] = self._learned_par[1][res[-1]]
        return out

The above class can be used as follows:

In [6]:
X_train = np.array([[0.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 1.0], [0.0, 0.0]])
y_train = np.array([1, 1, 0, 1, 0])

X_test = np.vstack((X_train, np.arange(0.1, 4.01, 0.1).reshape(-1, 2)))
y_test = np.hstack((y_train, np.zeros(X_test.shape[0] - y_train.shape[0])))

m = Memorizer()
m.fit(X_train, y_train)

# trained memorizer can predict the label for a sample represented as a list
X_list = [0.0, 1.0]
print(f'The prediction for input list {X_list} is {m.predict(X_list)[0]}')

# trained memorizer can predict the label for a sample represented as a numpy array
X_array = np.array([0.0, 0.0])
print(f'The prediction for input array {X_array} is {m.predict(X_array)[0]}')

prediction = m.predict(X_test)
print(f'par: {m.par}\nprediction: {prediction} positive predictions: {prediction.sum()}' 
      f'\nscore: {m.score(X_test, y_test)}')

# now we change the seed for the random number generator
m = Memorizer(rng_seed=2023)
m.fit(X_train, y_train)

prediction = m.predict(X_test)
print(f'par: {m.par}\nprediction: {prediction} positive predictions: {prediction.sum()}'
      f'\nscore: {m.score(X_test, y_test)}')


The prediction for input list [0.0, 1.0] is 1
The prediction for input array [0. 0.] is 0
par: {'_seed': 42, '_rng': Generator(PCG64) at 0x16F101951C0}
prediction: [0 1 0 1 0 0 0 0 1 1 1 0 0 0 1 1 1 1 0 0 0 1 0 0 0] positive predictions: 10
score: 0.64
par: {'_seed': 2023, '_rng': Generator(PCG64) at 0x16F101952A0}
prediction: [0 1 0 1 0 1 0 1 0 1 1 0 1 0 1 0 1 1 1 0 0 1 0 1 1] positive predictions: 14
score: 0.48


## Task 1: Implement perceptron (2 points)

Similarly, we can adapt our implementation of perceptron learning algorithm from a previous lab.
Complete the implementation below. The extended weight vector will be initialized as 
`np.asarray(init_weights, dtype=float)` - this enables that `init_weights` 
can be either list of floats, or numpy array, and the empty weigth vector 
can be tested using `self.par['weights'].size == 0)`.

In [7]:
class Perceptron(LearningAlgorithm):
    
    def __init__(self, init_weights=[], lr=1.0, max_epochs=1000):
        # Perceptron constructor
        # `init_weights` are the initial weights (including the bias) of the perceptron
        # `lr` is the learning rate
        # `max_epochs` is the maximum number of epochs
        super().__init__(weights = np.asarray(init_weights, dtype=float), lr=lr, max_epochs=max_epochs)

    def set_bias(self, X):
        X = np.asarray(X)
        if len(X.shape) == 1:
            return np.append(X, 1)
        else:
            return np.c_[X, np.ones(X.shape[0])]
        
    def predict(self, X: np.ndarray) -> np.array:
        # Compute the output of the perceptron.
        # Input `X` can be
        #  * a vector, i.e., one sample
        #  * or a two-dimensional array, where each row is a sample.
        # Returns
        #  * a vector with values 0/1 with the output of the perceptron 
        #    for all samples in `X`
        # Raises an exception if the weights are not initialized.
        X = self.set_bias(X)
        if not np.any(self.par['weights']):
            raise Exception('The weights are not initialized.')
        if len(X.shape) == 1:
            return [1] if np.dot(self.par['weights'], X) >= 0 else [0]
        else:
            return np.array([1 if np.dot(self.par['weights'], x) >= 0 else 0 for x in X])
    
    def partial_fit(self, X: np.ndarray, y: np.ndarray, lr=1.0) -> None:
        # perform one epoch perceptron learning algorithm 
        # on the training set `X` (two-dimensional numpy array of floats) with 
        # the desired outputs `y` (vector of integers 0/1) and learning rate `lr`.
        # If self.par['weights'] is empty, the weight vector is generated randomly.
        if not np.any(self.par['weights']):
            self.par['weights'] = np.random.rand(X.shape[1] + 1)
        for x, target in zip(X, y):
            x_bias = np.append(x, 1)
            self.par['weights'] += x_bias * lr * (target - self.predict(x))
            
    def fit(self, X: np.ndarray, y: np.ndarray, lr=None, max_epochs=None) -> int:
        # trains perceptron using perceptron learning algorithm
        # on the training set `X` (two-dimensional numpy array of floats) with 
        # the desired outputs `y` (vector of integers 0/1). 
        # If `self.par['weights'] is empty, the weight vector is generated randomly.
        # If the learning rate `lr` is `None`, `self.par['lr']` is used.
        # If `max_epochs` is `None`, `self.par['max_epochs']` is used. 
        # Returns the number of epochs used in the training (at most `max_epochs`).
        if max_epochs == None:
            max_epochs = self.par['max_epochs']
        if lr == None:
            lr = self.par['lr']
        for epochs in range (max_epochs):
            #print(self.par['weights'])
            self.partial_fit(X, y, lr)
            
            if self.score(X, y) == 1:
                print(self.par['weights'])
                return epochs + 1
        return max_epochs


A perceptron without weights cannot make predictions and must throw an exception (**0.5 points for this part**).

In [8]:
p = Perceptron(max_epochs=10)
try:
    p.predict([1,1])
except Exception as e:
    pass
else:
    raise AssertionError("Perceptron.predict with empty weights did not raise an exception")
 

Test the implementation of `Perceptron` carefully. It should pass all the tests below and some additional hidden tests (**1.5 points for this part**).

In [9]:
p = Perceptron(init_weights=[1,-2,1], max_epochs=10)
assert (p.predict(np.array([[6,3],[5,3],[1,1],[1,1.00001]])) == [1,1,1,0]).all()

X_list = [0.0, 1.0]
print(f"Prediction for the input list {X_list} is {p.predict(X_list)[0]}")
assert (p.predict(X_list)[0] == 0)

X_array = np.array([0.0, 0.0])
print(f"Prediction for the input array {X_array} is {p.predict(X_array)[0]}")
assert (p.predict(X_array)[0] == 1)

p = Perceptron(init_weights=[1,1,1], max_epochs=10)
X_train = np.array([[0.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 1.0]])
y_train = np.array([ 1, 0, 1, 0])

epochs = p.fit(X_train, y_train)
print(f"Training required {epochs} epoch(s)")
assert epochs == 2
print(f"Trained perceptron {p.par}")
#assert (p.par['weights'] == np.array([ 0., -1.,  0.])).all()
print(f"Score on the training set {p.score(X_train, y_train)}")
assert_approx_equal(p.score(X_train, y_train), 1.0)

X, y = make_classification(n_samples=100, n_features=2, n_informative=2, n_redundant=0, n_repeated=0, random_state=1234)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.3)
p = Perceptron([-1,1,2],0.5,5)
epochs = p.fit(X_train, y_train)
assert epochs==5
assert_approx_equal(p.score(X_test, y_test), 0.86666666)
assert_allclose(p.par['weights'], [0.04921529, 1.30631335, 0.        ])


Prediction for the input list [0.0, 1.0] is 0
Prediction for the input array [0. 0.] is 1
[ 0. -1.  0.]
Training required 2 epoch(s)
Trained perceptron {'weights': array([ 0., -1.,  0.]), 'lr': 1.0, 'max_epochs': 10}
Score on the training set 1.0


## Task 2: Implement k-fold cross-validation (4 points)

Then it is easy to implement the following function `cross_val` that estimates the difference between the error of the hypothesis
learned by a learning algorithm `learn_alg1` and the error of the hypotheses learned by a learning algorithm
`learn_alg2` using `k`-fold cross-validation on
patterns `X` with the desired outputs `y`. The function returns the estimated
difference of errors `delta` and estimated standard deviation `s` of this
estimator. **You should implement your own function using `numpy`, not use 
any implementation of cross-validation from any third-party library!**

* If the value of `shuffle` is `False`, then
  * the order of samples must not be changed before partitioning into folds for `k`-fold cross-validation, 
  * all folds should be continuous parts of `X`, and
  * the first fold (from the beginning of array `X`) must be used as the first test set and so on until the last fold is used as the last test set.
* If the value of `shuffle` is `True`, then 
  * the patterns from `X` should be assigned randomly into `k` folds (then, in general, calling `cross_val` repeatedly with the same parameters can result in different folds and different outputs).

_Notes:_ 
* _The sizes of the folds can differ by at most 1._
* _The function computes **estimates** of the error difference and standard deviation of the estimated difference of errors. Therefore, it should compute the **sample** standard deviation according to the formula $$\sigma = \sqrt{\frac{1}{k-1}\sum_{i=1}^k (\delta_i - \bar{\delta})^2}$$ where $\delta_i$ is the error difference if the $i$-th fold is used as the test set, and $\bar{\delta}$ is the average error difference on all folds._
* _Be aware that for each iteration of k-fold cross-validation, the learning algorithms must start from the same state of the learning algorithm. E.g., you can make a copy of a `learn_alg1` using `copy.deepcopy(learn_alg1)`._


In [10]:
import copy

def cross_val(k: int, learn_alg1: LearningAlgorithm, learn_alg2: LearningAlgorithm, 
              X: np.ndarray, y: np.ndarray, shuffle: bool = True, verbose=0) -> Tuple[float, float]:
    '''Estimates the difference between errors and the standard deviation of the difference for
    two learning algorithms.
    
        delta, std = cross_val(k, learn_alg1, learn_alg2, X, y, shuffle, verbose)
                               
    Args:
        k:           The number of folds used in k-fold cross-validation.
        learn_alg1:  The first learning algorithm.
        learn_alg2:  The second learning algorithm.
        name_apply1: The name of the function for applying the first learned function.
        X:           (2-d numpy array of floats): The training set; samples are rows.
        y:           (vector of integers 0/1): The desired outputs for the training samples.
        shuffle: If True, shuffle the samples into folds; otherwise, do not shuffle
                 the samples before splitting them into folds.
        verbose: If verbose == 0, it prints nothing. 
                 If verbose == 1, prints errors of both algorithms for each fold as a tuple.
                 If verbose == 2, for each fold, it prints 
                             the parameters of the first trained algorithm,
                             the parameters of the second trained algorithm, and 
                             the errors of both algorithms as an array with two elements 
        
    Returns:
        delta: The estimated difference: the error rate of the first algorithm minus 
            the error rate of the second algorithm computed using k-fold cross-validation.
        std: The sample standard deviation of the estimated difference of errors.            
    '''
    if shuffle:
        indices = np.random.permutation(X.shape[0])
        X = X[indices]
        y = y[indices]
    errors1 = []
    errors2 = []
    X_batch_size = X.shape[0]//k
    y_batch_size = y.shape[0]//k
    for i in range(k):
        learn_alg1_copy = copy.deepcopy(learn_alg1)
        learn_alg2_copy = copy.deepcopy(learn_alg2)
        X_train = np.concatenate((X[:i*X_batch_size], X[(i+1)*X_batch_size:]))
        y_train = np.concatenate((y[:i*y_batch_size], y[(i+1)*y_batch_size:]))
        X_test = X[i*X_batch_size:(i+1)*X_batch_size]
        y_test = y[i*y_batch_size:(i+1)*y_batch_size]
        learn_alg1_copy.fit(X_train, y_train)
        learn_alg2_copy.fit(X_train, y_train)
        errors1.append(1 - learn_alg1_copy.score(X_test, y_test))
        errors2.append(1 - learn_alg2_copy.score(X_test, y_test))
        if verbose == 2:
            print(f'fold {i+1}:')
            print(f'  {learn_alg1_copy.par}')
            print(f'  {learn_alg2_copy.par}')
            print(f'  errors: {errors1[-1]}, {errors2[-1]}')
        elif verbose == 1:
            print(f'fold {i+1}: {errors1[-1]}, {errors2[-1]}')
    delta = np.array(errors1) - np.array(errors2)
    std = np.sqrt((1/(k-1))*np.sum((delta - delta.mean())**2))
    
    return delta.mean(), std

In the next cell the implementatios of `cross_val` will be tested. The two visibe tests are followed with several hidden tests **(4 points for this part)**.

In [11]:
X, y = make_moons(n_samples=200, noise=0.2, random_state=5)

delta, sigma = cross_val(5, Perceptron(init_weights=[1,-2,1], max_epochs=10), 
             Perceptron(init_weights=[1,-2,1], max_epochs=100), X, y, shuffle=False, verbose=1)
print(f"Error difference: {delta} std: {sigma}")
assert_allclose((delta, sigma), (0.015, 0.03354101967))

X, y = make_blobs(n_samples=600, centers=2, n_features=2, random_state=4)

delta, sigma = cross_val(10, Perceptron(init_weights=[1,1,1], max_epochs=10), 
             Perceptron(init_weights=[1,1,1], max_epochs=100), X, y, shuffle=False, verbose=1)
print(f"Error difference: {delta} std: {sigma}")
assert_allclose((delta, sigma), (-0.0133333333, 0.045677344))


fold 1: 0.15000000000000002, 0.09999999999999998
fold 2: 0.22499999999999998, 0.17500000000000004
fold 3: 0.09999999999999998, 0.125
fold 4: 0.125, 0.125
fold 5: 0.275, 0.275
Error difference: 0.01499999999999999 std: 0.03354101966249685
fold 1: 0.09999999999999998, 0.15000000000000002
fold 2: 0.050000000000000044, 0.06666666666666665
fold 3: 0.15000000000000002, 0.09999999999999998
fold 4: 0.033333333333333326, 0.01666666666666672
fold 5: 0.09999999999999998, 0.1333333333333333
fold 6: 0.08333333333333337, 0.050000000000000044
fold 7: 0.033333333333333326, 0.033333333333333326
fold 8: 0.050000000000000044, 0.15000000000000002
fold 9: 0.050000000000000044, 0.033333333333333326
fold 10: 0.06666666666666665, 0.1166666666666667
Error difference: -0.01333333333333333 std: 0.04567734398020992


## Task 3: Implement computing of confidence interval (1 point)
When applying $k$-fold cross-validation, we will compute the confidence interval to estimate the difference of errors computed by the `cross_val()` function. Implement the following function.

In [12]:
from scipy.stats import t

def conf_interval(delta: float, sigma: float, conf_level: float, k: int) -> Tuple[float,float]:
    """Compute confidence interval for the estimated difference of errors d 
    with standard deviation s returned from cross_val().
    
        low, high = conf_inteval(delta, sigma, conf_level, k)
    
    Args:
        delta: The difference of errors computed by k-fold cross-validation.
        sigma: The standard deviation of the difference of errors computed 
            by k-fold cross-validation.
        conf_level: The confidence level. A value between 0 and 1.
        k: The number of folds used in k-fold cross-validation.
    """
    t_value = t.ppf((1 + conf_level)/2, k-1)
    low = delta - t_value * sigma / np.sqrt(k)
    high = delta + t_value * sigma / np.sqrt(k)
    return low, high

Function `conf-interval` must pass the following and several hidden tests **(1 point for this part)**.

In [13]:
assert_allclose(conf_interval(0.1, 0.03, 0.9, 10), (0.08260956,0.11739044))
assert_allclose(conf_interval(-0.1, 0.03, 0.95, 7), (-0.12774537,-0.07225463))

## Task 4: Compare learning algorithms using $k$-fold cross-validation (3 points)
<a id="compare_learn_alg"></a>

The above learning algorithms can be compared using the above function `cross_val` on the following datasets `dataset1` and `dataset2`.

In [14]:
dataset1 = np.genfromtxt('Data1.txt', delimiter=' ', dtype = float)
print(dataset1[0])
dataset2 = np.genfromtxt('Data2.txt', delimiter=' ', dtype = float)
print(dataset2[0])

[-0.04099436  1.71806508  1.        ]
[ 0.96478168 -2.06722166  0.71740289  3.22099464  2.9890881   0.        ]


### Compare Perceptron with Memorizer (1 point)
Compare `Perceptron([1, 1, -1], 1, 20)` and `Memorizer(rng_seed=2023)` on `dataset1` using 5-fold cross-validation with the confidence level `0.95`.

In [15]:
delta, sigma = cross_val(5,
                         Perceptron([1, 1, -1], 1, 20),
                         Memorizer(rng_seed=2023),
                         dataset1[:,:-1],
                         dataset1[:,-1], shuffle=True,
                         verbose=0)
print(f"Error difference: {delta} std: {sigma}")


Error difference: -0.35 std: 0.12247448713915891


Fill in the lower and upper limits of the corresponding confidence interval for the difference between the errors of the Perceptron and Memorizer:

In [16]:
low, high = conf_interval(delta, sigma, 0.95, 5)
print(f"Confidence interval: [{low}, {high}]")

Confidence interval: [-0.5020721613791638, -0.19792783862083616]


Is the error difference between the above two learning algorithms statistically significant? Explain your answer! An answer 'YES' or 'NO' will be graded with 0 points.

Yes, the difference is statistically significant because the confidence interval does not contains '0' which would be the value where there are no difference between the two algorithms.

### Compare two perceptrons (1 point)

Compare `Perceptron([1, -1, 1, -1, 1, -1], 1, 10)` and `Perceptron([1, -1, 1, -1, 1, -1], 1, 100)` on `dataset2` using 6-fold cross-validation with the confidence level `0.99`.

In [17]:
delta, sigma = cross_val(6, 
                        Perceptron([1, -1, 1, -1, 1, -1], 1, 10),
                        Perceptron([1, -1, 1, -1, 1, -1], 1, 100),
                        dataset2[:,:-1],
                        dataset2[:,-1],
                        shuffle=True,
                        verbose=0)
print(f"Error difference: {delta} std: {sigma}")

Error difference: -0.01833333333333335 std: 0.0842417157153549


Fill in the lower and upper limits of the corresponding confidence interval for the difference between the errors of the Perceptron and Memorizer:

In [18]:
low, high = conf_interval(delta, sigma, 0.99, 6)
print(f"Confidence interval: [{low}, {high}]")

Confidence interval: [-0.15700492562935545, 0.12033825896268875]


Is the error difference between the above two learning algorithms statistically significant? Explain your answer! An answer 'YES' or 'NO' will be graded with 0 points.

Not at all, the confidence interval contains 0 with a confidence level of 0.99 so this is not statiscally significant